In [1]:
import spacy

from medspacy.visualization import visualize_ent, MedspaCyVisualizerWidget

# Overview
In the last notebook, we used rule-based matching to extract mentions of treatments and problems from patient texts. One disadvantage of rule-based systems is that it can take a long time to manually curate all of the rules, particularly if the classes you're extracting are very broad and have many variants.

Another approach is **statistical NLP**. In this approach, we **annotate** a large corpus of text to identify the concepts we're interested in. Then we use those annotations as examples to train a machine learning model. In the machine learning module, we used ML to predict binary outcomes (0 or 1) for whether patients have conditions. In NER, a machine learning model predicts a label for each word in a sequence.

## i2b2
The NLP community hosts many **shared tasks**. These are competitions where multiple teams are given the same data and instructions and build different systems. These datasets include **gold standard annotations**, which are large sets of annotated examples.

To train a statistical model, I used data from the i2b2 2012 shared task: [**"Evaluating temporal relations in clinical text"**](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3756273/). This model was trained on data for the first subtask in the shared task, referred to in the challenge as **"Clinically relevant events"**. For the purpose of this module, I specifically restricted it to the following labels of **clinical concepts**:
- **Problems:** Diagnoses, signs, and symptoms
- **Tests:** Lab and vital measurements
- **Treatments:** Medications, procedures, and therapies


If you're interested in seeing how to train this model, see this notebook: [./appendix-train-i2b2-ner.ipynb](./appendix-train-i2b2-ner.ipynb). If you would like get access to the data for the challenge so you can actually train the model yourself, fill out this data access form on the i2b2 website: https://portal.dbmi.hms.harvard.edu/. Once you're approved, you can download the XML files and change the path variables in the notebook to read in the original data sources.



# Using our model
We use our model by using `spacy.load`, just like we would normally read in a default spaCy model. Make sure that you installed this in the [introduction](./00-introduction.ipynb) notebook. I named this model **"en_info_3700_i2b2_2012"**. The **"en"** stands for **"English"** (every spaCy model needs to start with its language), and the rest is named after the class and shared task.

In [2]:
nlp = spacy.load("en_info_3700_i2b2_2012")

/Users/alecchapman/opt/anaconda3/lib/python3.8/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_info_3700_i2b2_2012' (0.1.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.2). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


Our pipeline looks similar to the regular spaCy pipeline. But let's look at what labels are included in the `ner` component:

In [3]:
nlp.pipe_names

['tagger', 'parser', 'ner']

In [4]:
ner = nlp.get_pipe("ner")
ner.labels

('PROBLEM', 'TEST', 'TREATMENT')

We can now use our i2b2 model in the exact same way that we would use any other model. Let's call our clinical model on an example text.

In [5]:
text = "The patient was started on abx for his infection"
doc = nlp(text)

Let's look at what entities were extracted:

In [6]:
doc.ents

(abx, his infection)

In [7]:
visualize_ent(doc)

We can see that our model extracted two clinical entities: **"abx"**, a **treatment**, and **"his infection"**, a **problem**. 

Let's now process a few more documents and view the model output. We'll combine all of the following sentences and process them all as one `doc` so that we can view all the output simultaneously.

In [8]:
texts = [
    "87-year-old man with htn and end-stage renal disease.",
    "His wife recently died from end stage renal disease.",
    "The patient was started on abx for his infection.",
    "There is continued mild-to-moderate congestive heart failure. ",
    "The patient is s/p median sternotomy and right thoracotomy.",
    "The pt presents for ckd stage 4",
    "He previously had CKD stage 3.",
    "The patient presented to the emergency room with cough and fever, concern for infections.",
    "Patient prescribed coumadin for her atrial fibrillation.",
    "Patient prescribed coumadin for her AF.",
]


In [9]:
docs = list(nlp.pipe(texts))

In [10]:
w = MedspaCyVisualizerWidget(docs)

In [11]:
# idx = 0 # Change this to go through the docs
# visualize_ent(docs[idx])

### Discussion
Look at the output of the model above. Do you see any errors in the predictions? Are there any other limitations to the information extracted by the model?

## Another Example
### Discussion
Do you see any issues with the example below? Did the model predict correctly? Is there any other information we need to extract from the text?

In [12]:
doc = nlp("There is no evidence of pneumonia.")

In [13]:
visualize_ent(doc)

# Next Steps
We now have access to a statistical model which can identify clinical concepts reasonably well. However, we've also seen that that is sometimes not enough. Clinical documentation is full of instances where a concept is mentioned but is not necessarily present or relevant to the visit, such as the example above. In the next notebook, we'll see how to look for additional contextual information in a sentence and how we can use that to exclude concepts which are negated or not current.

[./04-context.ipynb](./04-context.ipynb)